## Web Crawling
- 웹 페이지에서 데이터를 수집하는 방법에 대해서 학습

### 웹크롤링 방법

#### 웹페이지의 종류
- 정적인 페이지 : 웹 브라우져에 화면이 한번 뜨면 이벤트에 의한 화면의 변경이 없는 페이지
- 동적인 페이지 : 웹 브라우져에 화면이 뜨고 이벤트가 발생하면 서버에서 데이터를 가져와 화면을 변경하는 페이지

#### requests 이용
- 받아오는 문자열에 따라 두가지 방법으로 구분
    - json 문자열로 받아서 파싱하는 방법 : 주로 동적 페이지 크롤링할때 사용
    - html 문자열로 받아서 파싱하는 방법 : 주로 정적 페이지 크롤링할때 사용
        
#### selenium 이용
- 브라우져를 직접 열어서 데이터를 받는 방법

#### 크롤링 방법에 따른 속도
- requests json > requests html > selenium

### Crawling Naver Stock Data
- 네이버 증권 사이트에서 주가 데이터 수집
- 수집할 데이터 : 일별 kospi, kosdaq 주가, 일별 환율(exchange rate) 데이터
- 데이터 수집 절차
    - 웹서비스 분석 : url
    - 서버에 데이터 요청 : request(url) > response : json(str)
    - 서버에서 받은 데이터 파싱(데이터 형태를 변경) : json(str) > list, dict > DataFrame

In [18]:
import warnings
warnings.filterwarnings('ignore') # 경고 문구 출력 x
import pandas as pd # 데이터 수집 후 csv 파일로 만들때
import requests

#### 1. 웹서비스 분석 : url
- pc 웹페이지가 복잡하면 mobile 웹페이지에서 수집

In [27]:
page_size, page = 60, 1
# url = 'https://m.stock.naver.com/api/index/KOSPI/price?pageSize=10&page=8'
url = f'https://m.stock.naver.com/api/index/KOSPI/price?pageSize={page_size}&page={page}'
url

'https://m.stock.naver.com/api/index/KOSPI/price?pageSize=60&page=1'

#### 2. 서버에 데이터 요청 : request(url) > response : json(str)
- response의 status code가 200이 나오는지 확인
- 403이나 500이 나오면 request가 잘못되거나 web server에서 수집이 안되도록 설정이 된것임
    - header 설정 또는 selenium 사용
- 200이 나오더라도 response 안에 있는 내용을 확인 > 확인하는 방법 : response.text

In [28]:
help(requests.get)
response = requests.get(url)
response

Help on function get in module requests.api:

get(url, params=None, **kwargs)
    Sends a GET request.
    
    :param url: URL for the new :class:`Request` object.
    :param params: (optional) Dictionary, list of tuples or bytes to send
        in the query string for the :class:`Request`.
    :param \*\*kwargs: Optional arguments that ``request`` takes.
    :return: :class:`Response <Response>` object
    :rtype: requests.Response



<Response [200]>

In [29]:
response.text[:50] # datatype : str

'[{"localTradedAt":"2024-03-07","closePrice":"2,646'

#### 3. 서버에서 받은 데이터 파싱(데이터 형태를 변경) : json(str) > list, dict > DataFrame

In [30]:
data = response.json() # str => list, dict
df = pd.DataFrame(data) # list, dict => DataFrame
columns = ['localTradedAt', 'closePrice']
df = df[columns]
df.head(2)

,localTradedAt,closePrice
0,2024-03-07,"2,646.56"
1,2024-03-06,"2,641.49"


#### 4. 함수로 만들기

In [37]:
def stock_price(code = 'KOSPI', page_size = 60, page = 1):
    # 1. 웹 서비스 분석 > URL
    url = f'https://m.stock.naver.com/api/index/{code}/price?pageSize={page_size}&page={page}'
    
    # 2. ruquest(URL) >> response(JSON) : JSON(str)
    response = requests.get(url)
    
    # 3. JSON(str) > list, dict > DataFrame : Data
    data = response.json()
    df = pd.DataFrame(data)
    return df[['localTradedAt', 'closePrice']]

In [39]:
stock_price()
df.tail(2)

,localTradedAt,closePrice
58,2023-12-08,"2,517.85"
59,2023-12-07,"2,492.07"


In [36]:
# f포맷팅 설명
balance = 1000
text = '저는 ' + str(balance) + '원가지고 있습니다.'
text = '저는 {}원 가지고 있습니다.'.format(balance)
text = f'저는 {balance}원 가지고 있습니다.'
text

'저는 1000원 가지고 있습니다.'

#### 5. 원달러 환율 데이터 수집 : 실습

In [47]:
# 1. 웹서비스분석 : URL
url = 'https://m.stock.naver.com/front-api/v1/marketIndex/prices?category=exchange&reutersCode=FX_USDKRW&page=1'
# 2. request(URL) > response(JSON) : JSON(str)
response = requests.get(url)
# 3. JSON(str) > list, dict > DataFrame : Data
data = response.json()['result']
df = pd.DataFrame(data)[['localTradedAt', 'closePrice']]
df.tail(2)

,localTradedAt,closePrice
8,2024-02-23,"1,332.50"
9,2024-02-22,"1,329.00"


#### 6. 시각화

In [49]:
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

In [50]:
import matplotlib.pyplot as plt
import seaborn as sns

In [52]:
# 데이터 수집


#### 7. 데이터 스케일링
- min max scaling


- $z = \frac{x_i - min(x)}{max(x) - min(x)} (0 \leqq z \leqq 1)$


- latex syntax : `https://jjycjnmath.tistory.com/117`

In [ ]:
from sklearn.preprocessing import minmax_scale

#### 8. 상관관계 분석
- 피어슨 상관계수(Pearson Correlation Coefficient)
- 두 데이터 집합의 상관도를 분석할때 사용되는 지표
- 상관계수의 해석
    - -1에 가까울수록 서로 반대방향으로 움직임
    - 1에 가까울수록 서로 같은방향으로 움직임
    - 0에 가까울수록 두 데이터는 관계가 없음